In [107]:
using CSV
using DataFrames
using Revise
using StatsBase
using StochasticPrograms
using Gurobi
using JuMP
using StatsPlots
cd("/Users/frankiecho/Library/CloudStorage/OneDrive-TheUniversityofQueensland/Documents/GitHub/koala-uncertainty/")
include("optim-functions.jl")

fcn_evaluate_solution (generic function with 1 method)

In [2]:
cost_df = CSV.read("data/spatial_predictions_10yr.csv", DataFrame);
kitl_index_full = CSV.read("data/kitl_prop_climate.csv", DataFrame);
stratified_samples = CSV.read("data/stratified_sample.csv", DataFrame);
climateProjList = ["CCCMA_R1", "CCCMA_R2", "CCCMA_R3", "CSIRO_R1", "CSIRO_R2", "CSIRO_R3", "ECHAM_R1", "ECHAM_R2", "ECHAM_R3", "MIROC_R1", "MIROC_R2", "MIROC_R3"];

In [3]:
names(cost_df)

33-element Vector{String}:
 "NewPropID"
 "KMR"
 "MosType"
 "LUSec"
 "LVAL"
 "AREA"
 "DMU"
 "DOU"
 "PTREE"
 "PGRASS"
 ⋮
 "SDAdopt"
 "SDWTA"
 "SDProp"
 "LowerAdopt"
 "LowerWTA"
 "LowerProp"
 "UpperAdopt"
 "UpperWTA"
 "UpperProp"

Calculate opportunity costs given an inflation rate

In [4]:
inflation_rate = 0.02;
t = 1:60;
tp = 1:6 # time periods
t_blocks = [(1:10) .+ (td-1)*10 for td in tp];
delta = (1+inflation_rate).^(t);
adoption = innerjoin(cost_df[:, [:NewPropID, :MeanWTA, :SDWTA, :MeanAdopt, :SDAdopt, :MeanProp, :SDProp, :AREA]], kitl_index_full[cmp.(kitl_index_full.climate_model,"Avg").==0, [:NewPropID]], on = :NewPropID);
cost_ts = (adoption.MeanWTA .* adoption.AREA ./ 10); # Cost per-year
cost_ts_full = [cost_ts .* d for d in delta];
cost_delta = mapreduce(permutedims, vcat, cost_ts_full)';
cost_blocks = mapreduce(permutedims, vcat, [sum(cost_delta[:,td], dims = 2) for td in t_blocks])';

In [5]:

tp = 1:size(cost_blocks,2)
kitl_threshold = 0.25; # cut-off for habitats deemed "high-quality"
ii = 1; # Stratified sample to use
kitl_index = innerjoin(kitl_index_full, adoption[:, [:NewPropID, :AREA, :MeanProp, :MeanAdopt]], on = :NewPropID);

In [74]:
cost_before = sum(cost_blocks[:, 3:(tt-1)], dims=2); # Cost start at 2020
cost_after = sum(cost_blocks[:, tt:tp[end]], dims=2);
t_symbols = [:t0, :t1, :t2, :t3, :t4, :t5, :t6, :t7]
tt = 4 # Time period when uncertainty is revealed

4

In [75]:
kitl_threshold_values = (kitl_index[:, t_symbols] .> kitl_threshold) .* kitl_index.AREA .* kitl_index.MeanProp;

Subset dataset to only stratified sampled cells

In [76]:
stratified_df = DataFrame(NewPropID = vec(stratified_samples[:, ii]))
cost_inputs = DataFrame(NewPropID = adoption.NewPropID, cost_before = vec(cost_before), cost_after = vec(cost_after));
cost_subset = innerjoin(stratified_df, cost_inputs, on = :NewPropID);
metric_inputs = DataFrame(NewPropID = kitl_index.NewPropID, climate_model = kitl_index.climate_model);
metric_inputs = hcat(metric_inputs, kitl_threshold_values);
metric_subset = innerjoin(stratified_df, metric_inputs, on = :NewPropID);
filter!(row -> cmp(row.climate_model, "Avg") != 0, metric_subset);
filter!(row -> row.NewPropID ∈ cost_subset.NewPropID, metric_subset);
filter!(row -> row.NewPropID ∈ metric_subset.NewPropID, cost_subset);

In [81]:
climateProjList = ["CCCMA_R1", "CCCMA_R2", "CCCMA_R3", "CSIRO_R1", "CSIRO_R2", "CSIRO_R3", "ECHAM_R1", "ECHAM_R2", "ECHAM_R3", "MIROC_R1", "MIROC_R2", "MIROC_R3"];
metric_subset

,NewPropID,climate_model,t0,t1,t2,t3,t4,t5
,Int64,String15,Float64,Float64,Float64,Float64,Float64,Float64
1,3039,CCCMA_R1,0.0,0.0,0.0,0.0,0.0,0.0
2,3039,CCCMA_R2,0.0,0.0,0.0,0.0,0.0,0.0
3,3039,CCCMA_R3,0.0,0.0,0.0,0.0,0.0,0.0
4,3039,CSIRO_R1,0.0,0.0,0.0,0.0,0.0,0.0
5,3039,CSIRO_R2,0.0,0.0,0.0,0.0,0.0,0.0
6,3039,CSIRO_R3,0.0,0.0,0.0,0.0,0.0,0.0
7,3039,ECHAM_R1,0.0,0.0,0.0,0.0,0.0,0.0
8,3039,ECHAM_R2,0.0,0.0,0.0,0.0,0.0,0.0
9,3039,ECHAM_R3,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
N = length(unique(metric_subset.NewPropID))
S = length(climateProjList)
M₁ = zeros(N, length(1:(tt-1)), S)
M₂ = zeros(N, length(tt:length(t_symbols)), S)
for t=1:length(t_symbols)
  for s=1:S
    metric_unstacked = unstack(metric_subset, :NewPropID, :climate_model, t_symbols[t])
    if (t < tt)
      M₁[:,t,:] = Matrix(metric_unstacked[:, climateProjList])
    else
      M₂[:,t-tt+1,:] = Matrix(metric_unstacked[:, climateProjList])
    end
  end
end

In [102]:
size(M₁)

(9537, 3, 12)

In [109]:
r = Realisation(cost_subset.cost_before, cost_subset.cost_after, M₁, M₂)

Realisation([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [35.29515852822115, 26.18636992317983, 32.03676101284311, 369.3841390058147, 381.5020683349195, 478.4022353340692, 396.37307405974104, 394.6432381247755, 260.19155587080905, 51.84901637646141  …  317.72742891077786, 2111.28750803746, 940.4204914731318, 1404.4953321986777, 161.36294877060345, 4755.142393027663, 11028.335636318545, 128.6265596701682, 389.59052836930425, 34.07014062701813], [0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0;;; … ;;; 0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0

In [114]:
[sum.(eachcol(M₂[:,:,s])) for s=1:S]

12-element Vector{Vector{Float64}}:
 [1918.8143800821963, 1305.6780141542145, 1208.5803611483432, 774.8736921318093, 873.5266274409778]
 [1840.4575312779002, 1147.1615821347275, 501.7452895747667, 2.92526086319456, 0.0]
 [1747.9725889999058, 1592.461709008147, 1127.233799824835, 755.216363105896, 428.3665770134107]
 [2177.9308800656213, 2111.05704858166, 1899.4046093771456, 1813.2751093489655, 1650.4633569607417]
 [2496.7653164389335, 2442.3449947019144, 2364.767599124096, 2107.505484866616, 1870.281319521435]
 [582.9928721946144, 1085.107006852796, 1288.627249757843, 1510.958171790325, 1844.333266866276]
 [2747.6748679843004, 2451.1285683007623, 0.0, 0.0, 1908.0533611753062]
 [2065.9587553521487, 1866.990300959405, 1762.3764347039255, 1633.221281511597, 1423.201446225884]
 [2907.012642611885, 2774.175157560566, 2830.3835634946236, 2738.3396221755415, 2950.802254263517]
 [2806.624904364392, 2424.149346978263, 2123.592027370344, 1474.4958766316036, 1268.564038768661]
 [2305.779862842521

In [111]:
solution_no_recourse = fcn_two_stage_opt_saa([r], add_recourse = false, terminate_recourse = false)
solution_add_recourse = fcn_two_stage_opt_saa([r], add_recourse = true, terminate_recourse = false)
solution_terminate_recourse = fcn_two_stage_opt_saa([r], add_recourse = false, terminate_recourse = true)
solution_full_recourse = fcn_two_stage_opt_saa([r], add_recourse = true, terminate_recourse = true)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-08


MathOptInterface.ResultIndexBoundsError{MathOptInterface.ObjectiveValue}: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [18]:
objective_value(solution_no_recourse.model)


71742.3701450581

In [19]:
objective_value(solution_full_recourse.model)

71720.491221524

In [20]:
objective_value(solution_add_recourse.model)

71742.3701450581

In [21]:

sum(value.(solution_full_recourse.model[:x]) .- value.(solution_full_recourse.model[:w])[:,10])

16.947677170788758

In [25]:
r.M₂' * value.(solution_no_recourse.model[:x])

12-element Vector{Union{Missing, Float64}}:
 7000.9677928874435
 7000.08224263148
 7000.892433471852
 7002.916334031763
 7004.144235863698
 7002.226535631552
 7001.512761354233
 7001.888360463729
 7003.585322681888
 7003.00522124162
 7002.1667130162505
 7002.892436153008